In [ ]:
import torch
import sys
import seqm
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure
seqm.seqm_functions.scf_loop.debug=True # print SCF steps 
seqm.seqm_functions.scf_loop.MAX_ITER=500 # MAX number of SCF iterations

DTYPE = torch.float64
torch.set_default_dtype(DTYPE)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
species = torch.as_tensor([[16,16],[22,22],[22,16],[35,17],[24,22]],dtype=torch.int64, device=device)

coordinates = torch.tensor([
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
    
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
    
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
    
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
    
        [[0.0000,    0.0,    0.0000],
         [0.0000,    1.2,    0.0000]],
                 ], device=device)


const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'PM6',  # AM1, MNDO, PM3, PM6, PM6_SP. PM6_SP is PM6 without d-orbitals. Effectively, PM6 for the first two rows of periodic table
                   'scf_eps' : 1.0e-5,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True,
                    "Hf_flag": True,
                    'scf_backward' : 1,
                    'UHF' : False, # open shell is currently not supported for PM6
                   }

molecules = Molecule(const, seqm_parameters, coordinates, species).to(device)
esdriver = Electronic_Structure(seqm_parameters).to(device)
esdriver(molecules)

# Total E

In [ ]:
molecules.Etot[0] # S2. MOPAC Etot = -333.8287 EV

In [ ]:
molecules.Etot[1] # Ti2. MOPAC Etot = -120.0651 EV

In [ ]:
molecules.Etot[2] # TiS. MOPAC Etot = -221.2701 EV

In [ ]:
molecules.Etot[3] # BrCl. MOPAC Etot = -458.5855 EV

In [ ]:
molecules.Etot[4] # CrTi. MOPAC Etot = -150.3163 EV  !!! Wrong nuclear rep parameters

# Nuclear repulsion

In [ ]:
molecules.Enuc[0] # S2. MOPAC Enuc = 290.8693 EV

In [ ]:
molecules.Enuc[1] # Ti2. MOPAC Enuc = 97.3923 EV

In [ ]:
molecules.Enuc[2] # TiS. MOPAC Enuc = 176.1518 EV

In [ ]:
molecules.Enuc[3] # BrCl. MOPAC Enuc = 391.1161 EV

In [ ]:
molecules.Enuc[4] # CrTi. MOPAC Enuc = 251.0941 EV  !!! Wrong nuclear rep parameters